In [2]:
import json
import pandas as pd
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk

In [3]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords_id = set(stopwords.words('indonesian'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\IMAM\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
with open("output/pandawa_dataset_v2.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [5]:
questions = [item['question'].lower() for item in data]
answers = [item['answer'].lower() for item in data]


In [6]:
def text_clean(corpus, daftar_tetap=None):
    """
    Bersihkan teks: hapus simbol, lowercase, kecuali kata khusus.
    """
    if daftar_tetap is None:
        daftar_tetap = []

    korpus_bersih = []
    for baris in corpus:
        kata_bersih = []
        for kata in baris.split():
            if kata not in daftar_tetap:
                # Hanya pertahankan huruf & angka
                kata_bersih.append(re.sub(r'[^a-zA-Z0-9]', '', kata).lower())
            else:
                kata_bersih.append(kata)
        korpus_bersih.append(' '.join(kata_bersih))
    return korpus_bersih

In [7]:
def hapus_stopword(corpus):
    """
    Hapus stopwords bahasa Indonesia.
    """
    korpus_bersih = []
    wh_words = {'siapa', 'apa', 'kapan', 'dimana', 'bagaimana', 'mengapa'}
    stop_id = stopwords_id - wh_words  # jangan hapus kata tanya
    for baris in corpus:
        kata_bersih = [kata for kata in baris.split() if kata not in stop_id]
        korpus_bersih.append(' '.join(kata_bersih))
    return korpus_bersih

In [8]:
daftar_tetap = []  
questions_clean = text_clean(questions, daftar_tetap)
questions_clean = hapus_stopword(questions_clean)

In [9]:
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(questions_clean)

In [10]:
def get_answer(user_input, threshold=0.2):
    input_clean = text_clean([user_input.lower()], daftar_tetap)
    input_clean = hapus_stopword(input_clean)
    input_vec = vectorizer.transform(input_clean)
    
    sim_scores = cosine_similarity(input_vec, X_tfidf)[0]
    best_score = sim_scores.max()
    best_index = sim_scores.argmax()

    if best_score < threshold:
        return "Maaf, saya tidak mengerti pertanyaan Anda."
    else:
        return answers[best_index]

In [11]:
def main(max_turns=5):
    username = input("Masukkan username Anda: ")
    print("Support Wayang: Halo! Selamat datang di Q&A Wayang. Ada yang bisa saya bantu?")
    
    for turn in range(max_turns):
        user_input = input(f"{username}: ")
        if user_input.lower() in ['bye', 'exit', 'quit']:
            print("Support Wayang: Sampai jumpa!")
            break
        else:
            jawaban = get_answer(user_input)
            print("Support Wayang:", jawaban)
    else:
        print(f"Support Wayang: Sesi selesai setelah {max_turns} pertanyaan. Terima kasih!")


In [13]:
if __name__ == "__main__":
    main()

Support Wayang: Halo! Selamat datang di Q&A Wayang. Ada yang bisa saya bantu?
Support Wayang: kisah ini menceritakan tentang arya gatutkaca yang bertapa di gunung argakelasa untuk menagih janji batara guru yang pernah diucapkannya dulu. kisah ini saya olah dari sumber rekaman wayang kulit dengan dalang ki nartosabdo, yang dipadukan dengan rekaman pentas ki manteb soedharsono, dengan perubahan seperlunya. kediri, 19 januari 2019 heri purwanto untuk daftar judul lakon wayang lainnya, silakan klik di sini arya gatutkaca. ------------------------------ ooo ------------------------------ prabu duryudana mendengar kabar arya gatutkaca bertapa di gunung argakelasa di kerajaan hastina, prabu duryudana memimpin pertemuan dihadap danghyang druna dari sokalima, patih sangkuni dari plasajenar, adipati karna dari awangga, dan raden kartawarma dari tirtatinalang. dalam pertemuan itu, patih sangkuni melaporkan bahwa arya gatutkaca saat ini tengah menduduki bekas padepokan milik bagawan bimasuci di gu